In [2]:
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

2023-04-05 17:35:43.115662: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#  use Image Data Generator to reshape images to avoid overfitting
# 1. / 255 -> scaling
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
# class mode = 'binary' because of the binary outcome. for more outcomes -> category
training_set = train_datagen.flow_from_directory('../datasets/cats_dogs_images/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


In [5]:
# Image Data Generator for the test set -> rescale only!
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('../datasets/cats_dogs_images/test_set',
    # use same target size and batch size as on training set
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [6]:
cnn = tf.keras.models.Sequential()

2023-04-05 14:59:29.506635: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Add layers

In [7]:
# add convolutional layers
# filters -> number of feature detectors
# kernel size -> dimensions 3x3
# input shape 64, 64 pixels + 3 of rgb channels
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# create pooling
# pool_size -> 2x2 pixels (picks the max value from the feature frame)
# strides -> shift by 2 pixels 
# paddind: 'valid' (no padding/default), 'same' use padding and don't calculate the vslues outside the square)
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

# add 2nd convolutional layer with the pool
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

# flattening
cnn.add(tf.keras.layers.Flatten())

Connect layers and add an output layer

In [8]:
# bigger number of units since we deal with images 64x64
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
# units = 1 and activation sigmoid (for binary output), softmax for many categories
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Compile and train

In [9]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
cnn.fit(x = training_set, validation_data=test_set, epochs=20)

Epoch 1/20
250/250 [==============================] - 41s 159ms/step - loss: 0.6636 - accuracy: 0.5978 - val_loss: 0.6037 - val_accuracy: 0.6850
Epoch 2/20
250/250 [==============================] - 34s 137ms/step - loss: 0.5826 - accuracy: 0.6950 - val_loss: 0.5787 - val_accuracy: 0.7075
Epoch 3/20
250/250 [==============================] - 33s 132ms/step - loss: 0.5454 - accuracy: 0.7212 - val_loss: 0.5307 - val_accuracy: 0.7420
Epoch 4/20
250/250 [==============================] - 34s 136ms/step - loss: 0.5162 - accuracy: 0.7448 - val_loss: 0.5112 - val_accuracy: 0.7500
Epoch 5/20
250/250 [==============================] - 34s 135ms/step - loss: 0.4898 - accuracy: 0.7582 - val_loss: 0.4778 - val_accuracy: 0.7780
Epoch 6/20
250/250 [==============================] - 38s 151ms/step - loss: 0.4579 - accuracy: 0.7780 - val_loss: 0.4577 - val_accuracy: 0.7900
Epoch 7/20
250/250 [==============================] - 35s 140ms/step - loss: 0.4479 - accuracy: 0.7870 - val_loss: 0.5093 - val_ac

In [14]:
cnn

In [24]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [29]:
test1 = tf.keras.utils.load_img('../datasets/cats_dogs_images/imgs/1.jpg', target_size=(64, 64))
test1 = tf.keras.utils.img_to_array(test1)
test1 = np.expand_dims(test1, axis=0)
result1 = cnn.predict(test1)

1/1 [==============================] - 0s 21ms/step


In [30]:
result1

array([[1.]], dtype=float32)

In [22]:
result1[0][0]

1.0

In [23]:
test2 = tf.keras.utils.load_img('../datasets/cats_dogs_images/imgs/2.jpg', target_size=(64, 64))
test2 = tf.keras.utils.img_to_array(test2)
test2 = np.expand_dims(test2, axis=0)
result2 = cnn.predict(test2)

1/1 [==============================] - 0s 20ms/step


In [24]:
if result2 == 0:
    print('Cat')
else:
    print('Dog')

Cat


In [12]:
def get_result(cnn, path:str):
    '''
    the function returns prediction cat or dog
    '''
    # {'cats': 0, 'dogs': 1}
    output = training_set.class_indices
    
    img = tf.keras.utils.load_img(path, target_size=(64, 64))
    img = tf.keras.utils.img_to_array(img)
    # add a dimention for a batch. put it on the 1st place axis=0
    img = np.expand_dims(img, axis=0)
    # alternatively (from Keras API docs)
    # img = np.array([img])  # Convert single image to a batch.
    res = cnn.predict(img)[0][0].astype(np.uint8)
    final = ''
    for key in output:
        if output[key] == res:
            final = key
    
    return final.title()

In [37]:
def get_result_corrected(cnn, path:str):
    '''
    the function returns prediction cat or dog
    '''
    # {'cats': 0, 'dogs': 1}
    output = training_set.class_indices
    
    img = tf.keras.utils.load_img(path, target_size=(64, 64))
    img = tf.keras.utils.img_to_array(img)
    # add a dimention for a batch. put it on the 1st place axis=0
    img = np.expand_dims(img, axis=0)
    # alternatively (from Keras API docs)
    # img = np.array([img])  # Convert single image to a batch.

    # corrected version with normalized input image and probability return
    res = cnn.predict(img / 255.0)[0][0]
    if res > 0.5:
        res = 1
    else:
        res = 0
    final = ''
    for key in output:
        if output[key] == res:
            final = key
    
    return final.title()

#### Save the model if check if it is correct

In [4]:
import pickle
import joblib
import os
import json

from tensorflow.keras.models import model_from_json

directory = 'model_cnn'
if not os.path.isdir(directory):
    os.mkdir(directory)

In [16]:
cnn.save('models')

INFO:tensorflow:Assets written to: models/assets


INFO:tensorflow:Assets written to: models/assets


In [18]:
cnn_loaded = tf.keras.models.load_model('models')

In [33]:
img1 = '../datasets/cats_dogs_images/imgs/1.jpg' # DOG
get_result(cnn=cnn_loaded, path=img1)
# saved and loaded model returned wrong result :o)

1/1 [==============================] - 0s 21ms/step


'Dogs'

In [34]:
get_result(cnn, img1)

1/1 [==============================] - 0s 20ms/step


'Dogs'

In [38]:
get_result_corrected(cnn_loaded, img1)

1/1 [==============================] - 0s 20ms/step


'Dogs'

In [9]:
directory = 'models'
if not os.path.isdir(directory):
    os.mkdir(directory)

file_pickle = directory + '/cnn.pkl'
file_json = directory + '/cnn.json'
weight_path = directory + '/cnn.h5'

In [64]:
# serialize model to JSON
model_json = cnn.to_json()
with open(file_json, "w") as json_file:
    json_file.write(model_json)
# save weight mandatory!!!
# without weights only layers are saved
cnn.save_weights(weight_path)

In [65]:
# load json and create model
json_file = open(file_json, 'r')
model_json = json_file.read()
json_file.close()
cnn_json = model_from_json(model_json)
cnn_json.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn_json.load_weights(weight_path)

In [52]:
model_json

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 64, 64, 3], "dtype": "float32", "sparse": false, "ragged": false, "name": "conv2d_input"}}, {"class_name": "Conv2D", "config": {"name": "conv2d", "trainable": true, "dtype": "float32", "batch_input_shape": [null, 64, 64, 3], "filters": 32, "kernel_size": [3, 3], "strides": [1, 1], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1, 1], "groups": 1, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "MaxPooling2D", "config": {"name": "max_pooling2d", "trainable": true, "dtype": "float32", "pool_size": [2, 2], "padding": "valid", "strides": [2, 2], "d

In [48]:
cnn_json

In [68]:
# without saved weights the model returned Cats
get_result(cnn_json, img1)

1/1 [==============================] - 0s 73ms/step


'Dogs'

In [69]:
get_result(cnn_json, img1)

1/1 [==============================] - 0s 21ms/step


'Dogs'

In [51]:
get_result_corrected(cnn_json, img1)

1/1 [==============================] - 0s 23ms/step


'Cats'

To pickle -> write binary

In [6]:
cnn = tf.keras.models.load_model('model_cnn')

2023-04-05 17:37:50.639589: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
cnn

In [15]:

pickle.dump(cnn, open(file_pickle, 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......flatten
.........vars
......max_pooling2d
.........vars
......max_pooling2d_1
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-04-05 17:40:12         3861
metadata.json                                  2023-04-05 17:40:12           64
variables.h5                                   2023-04-05 17:40:12      3280816


In [16]:
pickle.dump(cnn, open(file_pickle, 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......flatten
.........vars
......max_pooling2d
.........vars
......max_pooling2d_1
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-04-05 17:40:13         3861
metadata.json                                  2023-04-05 17:40:13           64
variables.h5                                   2023-04-05 17:40:13      3280816


In [18]:
# throws an error when I save a reconstructed model
# cnn_pickle = pickle.load(open(file_pickle, 'rb'))